In [1]:
import torch
import json
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForVision2Seq, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

D:\Andrew\GeekBrains\Python\AITableProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_device():
    """Определение устройства: TPU → GPU → CPU"""
    if _has_tpu:
        try:
            device = xm.xla_device()
            num_cores = os.environ.get("TPU_NUM_CORES", "8")
            print(f"Используем TPU (ядра: {num_cores})")
            return device, "tpu"
        except Exception as e:
            print("TPU найден, но ошибка при инициализации:", e)

    if torch.cuda.is_available():
        print("GPU доступен, используем CUDA")
        return torch.device("cuda"), "gpu"

    print("Используем CPU")
    return torch.device("cpu"), "cpu"

In [3]:
# TPU-библиотека
try:
    import torch_xla.core.xla_model as xm
    _has_tpu = True
except ImportError:
    _has_tpu = False

In [4]:
# ---- выбор устройства ----
device, device_type = get_device()

# ---- dtype ----
if device_type == "tpu":
    torch_dtype = torch.bfloat16   # только bf16
elif device_type == "gpu":
    torch_dtype = torch.float16    # на GPU можно fp16
else:
    torch_dtype = torch.float32    # на CPU обычный float32

Используем CPU


In [5]:
# Модель и токенизатор. Преобразовываем текст (слова) в токены
model_name = "Qwen/Qwen2-VL-2B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForVision2Seq.from_pretrained( #AutoModelForVision2Seq автоматический класс из библиотеки transformers, который подбирает правильный токенизатор для данной модели, from_pretrained загружает готовый токенизатор по имени модели
    model_name,
    torch_dtype=torch_dtype,
    trust_remote_code=True
).to(device)

D:\Andrew\GeekBrains\Python\AITableProject\.venv\Lib\site-packages\transformers\models\auto\modeling_auto.py:2214: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


In [6]:
# Добавляем LoRA (Low-Rank Adaptation) техника дообучения больших моделей с малым числом параметров
peft_config = LoraConfig(
    r=16,  # Ранг матриц Чем больше r, тем больше добавляется обучаемых параметров и тем выше точность адаптации, но больше нагрузка на память.
    lora_alpha=32, # Контролирует, насколько сильно добавочные веса (из LoRA) влияют на модель. регулирует «усиление» новых параметров
    target_modules=["q_proj", "v_proj"],  # Целевые слои для Qwen. Слои модели, куда будут вставлены LoRA-адаптеры
    lora_dropout=0.05, #dropout некоторых связей во время обучения
    bias="none", # Определяет, будут ли дообучаться смещения (bias terms). "all" или "lora_only"
    task_type="CAUSAL_LM" #Тип задачи. "CAUSAL_LM" = причинно-следственная языковая модель (autoregressive language model, т.е. предсказание следующего токена)
)
model = get_peft_model(model, peft_config) # Оборачивает исходную модель в оболочку PEFT (Parameter-Efficient Fine-Tuning). В выбранные модули (q_proj, v_proj) вставляются LoRA-адаптеры. Обучаются только
# LoRA-параметры, а остальные веса модели остаются замороженными.
model.print_trainable_parameters()  # Показывает, сколько параметров будут обучаться

trainable params: 2,179,072 || all params: 2,211,164,672 || trainable%: 0.0985


In [ ]:
# Загружаем наш датасет
data_files = "./datasets/data/train.jsonl"
dataset = load_dataset("json", data_files=data_files, split="train")

В transformers при обучении языковых моделей (Causal LM)

input_ids → токены, которые подаются на вход модели.

labels → эталонные ответы, которые модель должна предсказывать.

Модель обучается так:
На каждом шаге она получает часть input_ids и должна предсказать следующий токен из labels.

Если мы обучаем авторегрессию (CAUSAL LM), то:

вход (input_ids) и эталон (labels) совпадают.

Разница в том, что внутри модели логиты на позиции i используются для предсказания токена на позиции i+1.

Но часто нам нужно, чтобы модель училась только на ответе, а не на самом вопросе.
Для этого делают маскирование промпта в labels:

Все токены промпта заменяют на -100.

В transformers это специальное значение, означающее «не учитывать в лоссе».


Для нашей задачи

Формат входа и выхода должен быть строго определён.

Лучше всего представлять таблицу в текстовом виде:

вход: строка с разделителями,

выход: та же строка, но с новыми колонками.

labels должны обучать модель именно на генерацию выходной таблицы, а не на повторение входа.

То есть labels должны покрывать только output, а prompt (т.е. сам «вопрос») нужно замаскировать (-100 в labels).

In [ ]:
def tokenize(example):
    # Формируем промпт: даём задачу модели
    prompt = f"Преобразуй данные:\n{json.dumps(example['input'], ensure_ascii=False)}\n\nВыход (только JSON):\n:\n"

    # Выход приводим к строке JSON
    output = json.dumps(example["output"], ensure_ascii=False)

    # Соединяем в один текст
    full_text = prompt + output

    # Токенизируем весь текст (и промпт, и ответ)
    tokenized = tokenizer(
        full_text,
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    # labels = копия input_ids
    labels = tokenized["input_ids"].copy()

    # Длина промпта (чтобы не считать его в лоссе)
    prompt_len = len(tokenizer(prompt)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len

    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

In [ ]:
# Параметры обучения

training_args = TrainingArguments(
    per_device_train_batch_size=1 if device_type != "gpu" else 2,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=5,
    logging_dir="./logs",
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    fp16=True if device_type == "gpu" else False,
    bf16=True if device_type == "tpu" else False,
    tpu_num_cores=1 if device_type == "tpu" else None
)

In [ ]:
# collator будет автоматически выравнивать длину примеров в батче
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)
trainer.train()

In [ ]:
# Сохраняем только адаптеры LoRA
model.save_pretrained("./qwen-lora-adapters")
tokenizer.save_pretrained("./qwen-lora-adapters")

In [12]:
# Для инференса: Загружаем базовую модель и добавляем адаптеры
from transformers import pipeline
from peft import PeftModel
import json

base_model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch_dtype,
    trust_remote_code=True
).to(device)

model = PeftModel.from_pretrained(
    base_model,
    "./qwen-lora-adapters",
    torch_dtype=torch_dtype,
).to(device)

test_input = "Отвертка 7018-2025 ГОСТ 1050-2023;Шт.;4140715;15"
test_input_list = test_input.split(";")

prompt = f"""Преобразуй данные:\n{json.dumps(test_input_list, ensure_ascii=False)}\nВыход (только JSON):\n
"""

# ---- настройка пайплайна ----
if device_type == "gpu":
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0   # GPU
    )
    output = pipe(prompt, max_new_tokens=200)
    print("==== Сырой вывод ====")
    print(output)
    print("\n==== Только ответ ====")
    print(output[0]["generated_text"].strip())

elif device_type == "cpu":
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=-1  # CPU
    )
    output = pipe(prompt, max_new_tokens=200)
    print("==== Сырой вывод ====")
    print(output)
    print("\n==== Только ответ ====")
    print(output[0]["generated_text"].strip())

elif device_type == "tpu":
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.35it/s]
Device set to use cpu


==== Сырой вывод ====
[{'generated_text': 'Преобразуй данные:\n["Отвертка 7018-2025 ГОСТ 1050-2023", "Шт.", "4140715", "15"]\nВыход (только JSON):\n\n {"Наименование": "Отвертка 7018-2025 ГОСТ 1050-2023", "Единица измерения": "шт.", "Количество": "4140715", "Техническое задание": "Артикул: 1050000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'}]

==== Только ответ ====
Преобразуй данные:
["Отвертка 7018-2025 ГОСТ 1050-2023", "Шт.", "4140715", "15"]
Выход (только JSON):

 {"Наименование": "Отвертка 7018-2025 ГОСТ 1050-2023", "Единица измерения": "шт.", "Количество": "4140715", "Техническое задание": "Артикул: 1050000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
